<a href="https://colab.research.google.com/github/TerrenceAm22/DS-Unit-1-Sprint-1-Data-Wrangling-and-Storytelling/blob/master/Copy_of_LS_DS_121_Statistics_Probability_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 1 Sprint 2 Assignment 1*

# Apply the t-test to real data

Your assignment is to determine which issues have "statistically significant" differences between political parties in this [1980s congressional voting data](https://archive.ics.uci.edu/ml/datasets/Congressional+Voting+Records). The data consists of 435 instances (one for each congressperson), a class (democrat or republican), and 16 binary attributes (yes or no for voting for or against certain issues). Be aware - there are missing values!

Your goals:

1. Load and clean the data (or determine the best method to drop observations when running tests)
2. Using hypothesis testing, find an issue that democrats support more than republicans with p < 0.01
3. Using hypothesis testing, find an issue that republicans support more than democrats with p < 0.01
4. Using hypothesis testing, find an issue where the difference between republicans and democrats has p > 0.1 (i.e. there may not be much of a difference)

Note that this data will involve *2 sample* t-tests, because you're comparing averages across two groups (republicans and democrats) rather than a single group against a null hypothesis.

In [14]:
### YOUR CODE STARTS HERE

import numpy as np
import pandas as pd
from scipy import stats

data = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/voting-records/house-votes-84.data')
print(data)

     republican  n  y n.1 y.1 y.2 y.3 n.2 n.3 n.4 y.4  ? y.5 y.6 y.7 n.5 y.8
0    republican  n  y   n   y   y   y   n   n   n   n  n   y   y   y   n   ?
1      democrat  ?  y   y   ?   y   y   n   n   n   n  y   n   y   y   n   n
2      democrat  n  y   y   n   ?   y   n   n   n   n  y   n   y   n   n   y
3      democrat  y  y   y   n   y   y   n   n   n   n  y   ?   y   y   y   y
4      democrat  n  y   y   n   y   y   n   n   n   n  n   n   y   y   y   y
..          ... .. ..  ..  ..  ..  ..  ..  ..  ..  .. ..  ..  ..  ..  ..  ..
429  republican  n  n   y   y   y   y   n   n   y   y  n   y   y   y   n   y
430    democrat  n  n   y   n   n   n   y   y   y   y  n   n   n   n   n   y
431  republican  n  ?   n   y   y   y   n   n   n   n  y   y   y   y   n   y
432  republican  n  n   n   y   y   y   ?   ?   ?   ?  n   y   y   y   n   y
433  republican  n  y   n   y   y   y   n   n   n   y  n   y   y   y   ?   n

[434 rows x 17 columns]


In [19]:

N = 434

a = np.random.randn(N)+2
b = np.random.randn(N)

var_a  = a.var(ddof=1)
var_b  = b.var(ddof=1)

s = np.sqrt((var_a + var_b)/2)
s

t = (a.mean() - b.mean())/(s*np.sqrt(2/N))

df = 2*N - 2

p = 1 - stats.t.cdf(t,df=df)

print("t = "+ str(t))
print("p ="+str(2*p))


t2, p2 = stats.ttest_ind(a,b)
print("t="+str(t2))
print("p="+ str(p2))


t = 29.21251714069648
p =0.0
t=29.212517140696487
p=4.119643539256044e-131


## Stretch Goals:

1. Refactor your code into functions so it's easy to rerun with arbitrary variables
2. Work on Performing a T-test without using Scipy in order to get "under the hood" and learn more thoroughly about this topic.
### Start with a 1-sample t-test
 - Establish the conditions for your test 
 - [Calculate the T Statistic](https://blog.minitab.com/hs-fs/hubfs/Imported_Blog_Media/701f9c0efa98a38fb397f3c3ec459b66.png?width=247&height=172&name=701f9c0efa98a38fb397f3c3ec459b66.png) (You'll need to omit NaN values from your sample).
 - Translate that t-statistic into a P-value. You can use a [table](https://www.google.com/search?q=t+statistic+table) or the [University of Iowa Applet](https://homepage.divms.uiowa.edu/~mbognar/applets/t.html)

 ### Then try a 2-sample t-test
 - Establish the conditions for your test 
 - [Calculate the T Statistic](https://lh3.googleusercontent.com/proxy/rJJ5ZOL9ZDvKOOeBihXoZDgfk7uv1YsRzSQ1Tc10RX-r2HrRpRLVqlE9CWX23csYQXcTniFwlBg3H-qR8MKJPBGnjwndqlhDX3JxoDE5Yg) (You'll need to omit NaN values from your sample).
 - Translate that t-statistic into a P-value. You can use a [table](https://www.google.com/search?q=t+statistic+table) or the [University of Iowa Applet](https://homepage.divms.uiowa.edu/~mbognar/applets/t.html)

 ### Then check your Answers using Scipy!